# Advanced ensembling techniques

In this project I am going to work with a dataset based on the following data:

https://archive.ics.uci.edu/ml/datasets/Online+News+Popularity

_Citation:_

* _K. Fernandes, P. Vinagre and P. Cortez. A Proactive Intelligent Decision Support System for Predicting the Popularity of Online News. Proceedings of the 17th EPIA 2015 - Portuguese Conference on Artificial Intelligence, September, Coimbra, Portugal._

The dataset contains the information about the internet news articles. I am going to predict a number of shares of the news article (target column: `shares`). The information about the features is available through the link above.

In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor, early_stopping
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [2]:
df = pd.read_csv('data/OnlineNewsPopularity.csv')

In [3]:
df.head()

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,6.0,73.0,0.916667,1.0,0.980392,6.0,0.0,1.0,0.0,4.917808,...,0.150000,0.5,-0.200000,-0.2,-0.200000,0.000000,0.000000,0.500000,0.000000,878
1,10.0,1280.0,0.407072,1.0,0.572592,53.0,2.0,10.0,1.0,4.496875,...,0.033333,1.0,-0.396465,-1.0,-0.050000,0.000000,0.000000,0.500000,0.000000,1100
2,9.0,576.0,0.535088,1.0,0.693182,9.0,5.0,1.0,1.0,4.850694,...,0.100000,0.9,-0.352778,-1.0,-0.100000,0.000000,0.000000,0.500000,0.000000,872
3,9.0,210.0,0.583732,1.0,0.729927,6.0,2.0,1.0,0.0,4.804762,...,0.062500,0.5,-0.250000,-0.4,-0.100000,0.454545,0.136364,0.045455,0.136364,5000
4,13.0,1578.0,0.429864,1.0,0.624595,34.0,24.0,11.0,0.0,4.593790,...,0.033333,1.0,-0.332037,-1.0,-0.071429,0.000000,0.000000,0.500000,0.000000,1600


First of all, we can check if there are any missing values in the data.

In [4]:
df.isna().sum().sum()

0

Let's separate the target from the dataframe with features.

Next, let's split the data into train/val/test sets in the ratio 60:20:20. I will use train set to train the models, val set to validate them and test set to calculate the final error of the blend. So, test set will be a completely unseen data.

In [5]:
X = df.drop('shares', axis=1)
y = df['shares']

In [6]:
X_train, X_val_test, y_train, y_val_test = train_test_split(
    X, y, test_size=0.4, random_state=13)

X_val, X_test, y_val, y_test = train_test_split(
    X_val_test, y_val_test, test_size=0.5, random_state=13)


---

Now let's train the first model - XGBoost. A link to the documentation: https://xgboost.readthedocs.io/en/latest/

I will use Scikit-Learn Wrapper interface for XGBoost (and the same logic applies to the following LightGBM and CatBoost models). Here, we have a regression task - hence I will use `XGBRegressor`. The parameters of `XGBRegressor`: https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRegressor

The main list of XGBoost parameters: https://xgboost.readthedocs.io/en/latest/parameter.html

In [7]:
xgb_reg = XGBRegressor(objective='reg:squarederror',
                       n_estimators=200,
                       max_depth=5,
                       learning_rate=0.01,
                       random_state=13)

xgb_reg.fit(X_train, y_train)
y_pred_val_xgb = xgb_reg.predict(X_val)

round(np.sqrt(mean_squared_error(y_val, y_pred_val_xgb)), 5)

10329.20768

I have decided to build 200 trees in the model. However, it is hard to understand whether it is a good decision.

During the training process, it is possible to stop constructing the ensemble if the validation error does not decrease anymore. Let's train the same the same XGBoost model with `eval_set=[(X_val, y_val)]` (to evaluate the boosting model after building a new tree) and `early_stopping_rounds=50`.

In [8]:
xgb_reg = XGBRegressor(objective='reg:squarederror',
                       n_estimators=200,
                       max_depth=5,
                       learning_rate=0.01,
                       early_stopping_rounds=50,
                       random_state=13)

xgb_reg.fit(X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=False)

y_pred_val_xgb = xgb_reg.predict(X_val)

round(np.sqrt(mean_squared_error(y_val, y_pred_val_xgb)), 5)

8890.23464

Notes on parameter tuning: https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html

Here, I tried to tune some parameters of the algorithm.

In [9]:
xgb_reg = XGBRegressor(objective='reg:squarederror',
                         n_estimators=5000,
                         max_depth=4,
                         learning_rate=0.001,
                         gamma=1,
                         subsample=0.5,
                         early_stopping_rounds=500,
                         random_state=13)

xgb_reg.fit(X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=False)

y_pred_val_xgb = xgb_reg.predict(X_val)

round(np.sqrt(mean_squared_error(y_val, y_pred_val_xgb)), 5)

8692.77417

Let's calculate feature importances according to the trained model and find the most important feature.

In [10]:
max(zip(X_train.columns, xgb_reg.feature_importances_), key=lambda k: k[1])[0]

'data_channel_is_bus'


---

Let's move to LightGBM. I will work with `LGBMRegressor`.

LGBMRegressor parameters: https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html#lightgbm.LGBMRegressor

The main list of LightGBM parameters: https://lightgbm.readthedocs.io/en/latest/Parameters.html

In [11]:
lgbm_reg = LGBMRegressor(objective='regression',
                         max_depth=5,
                         learning_rate=0.01,
                         n_estimators=200,
                         random_state=13)

lgbm_reg.fit(X_train, y_train,
             eval_set=[(X_val, y_val)],
             eval_metric='rmse',
             callbacks=[early_stopping(stopping_rounds=50, verbose=False)])

y_pred_val_lgbm = lgbm_reg.predict(X_val)

round(np.sqrt(mean_squared_error(y_val, y_pred_val_lgbm)), 5)

8451.2859

The speed of the algorithm is noticeably faster in comparison with the speed of XGBoost model.

Notes on parameter tuning: https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

Here, I tried to tune some parameters of the algorithm.

In [12]:
lgbm_reg = LGBMRegressor(objective='regression',
                         boosting_type='goss',
                         max_depth=3,
                         learning_rate=0.001,
                         n_estimators=5000,
                         lambda_l2=1.0,
                         num_threads=24, # https://github.com/microsoft/LightGBM/issues/5441
                         random_state=13)

lgbm_reg.fit(X_train, y_train,
             eval_set=[(X_val, y_val)],
             eval_metric='rmse',
             callbacks=[early_stopping(stopping_rounds=500, verbose=False)])

y_pred_val_lgbm = lgbm_reg.predict(X_val)

round(np.sqrt(mean_squared_error(y_val, y_pred_val_lgbm)), 5)

[LightGBM] [Warning] num_threads is set=24, n_jobs=-1 will be ignored. Current value: num_threads=24
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0


8421.39852

Let's calculate feature importances according to the trained model and find the most important feature.

In [13]:
max(zip(lgbm_reg.feature_name_, lgbm_reg.feature_importances_), key=lambda k: k[1])[0]

'self_reference_min_shares'

Since some features are not important for the model, we can drop them in order to try to construct a better model which does not consider them at all.

Let's obtain new train and validation sets without the features with LightGBM importance less than 10 and train the same model on the new train set in the same manner.

In [14]:
important_features = list(name for name, importance in zip(lgbm_reg.feature_name_, lgbm_reg.feature_importances_) if importance >= 10)

X_train_new = X_train[important_features]
X_val_new = X_val[important_features]

In [15]:
lgbm_reg.fit(X_train_new, y_train,
             eval_set=[(X_val_new, y_val)],
             eval_metric='rmse',
             callbacks=[early_stopping(stopping_rounds=500, verbose=False)])

y_pred_val_lgbm = lgbm_reg.predict(X_val_new)

round(np.sqrt(mean_squared_error(y_val, y_pred_val_lgbm)), 5)

[LightGBM] [Warning] num_threads is set=24, n_jobs=-1 will be ignored. Current value: num_threads=24
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0


8422.73009


---

Let's move to CatBoost. I will work with `CatBoostRegressor`.

Info about `CatBoostRegressor`: https://catboost.ai/docs/concepts/python-reference_catboostregressor.html

CatBoost parameters: https://catboost.ai/docs/concepts/python-reference_parameters-list.html

In [16]:
cat_reg = CatBoostRegressor(loss_function='RMSE',
                            iterations=200,
                            learning_rate=0.01,
                            max_depth=5,
                            random_state=13)

cat_reg.fit(X_train, y_train,
            eval_set=[(X_val, y_val)],
            early_stopping_rounds=50,
            verbose=False)

y_pred_val_cat = cat_reg.predict(X_val)

round(np.sqrt(mean_squared_error(y_val, y_pred_val_cat)), 5)

8485.01086

The speed of the algorithm is even faster in comparison with the speed of XGBoost and LightGBM models.

Notes on parameter tuning: https://catboost.ai/docs/concepts/parameter-tuning.html

Here, I tried to tune some parameters of the algorithm.

In [17]:
cat_reg = CatBoostRegressor(loss_function='RMSE',
                            n_estimators=5000,
                            learning_rate=0.001,
                            max_depth=9,
                            random_state=13)

cat_reg.fit(X_train, y_train,
            eval_set=[(X_val, y_val)],
            early_stopping_rounds=500,
            verbose=False)

y_pred_val_cat = cat_reg.predict(X_val)

round(np.sqrt(mean_squared_error(y_val, y_pred_val_cat)), 5)

8465.44037

Let's calculate feature importances according to the trained model and find the most important feature.

In [18]:
max(zip(cat_reg.feature_names_, cat_reg.feature_importances_), key=lambda k: k[1])[0]

'kw_avg_avg'


---

Finally, let's take a `Lasso` model from `sklearn` and train it on the train set.

In [19]:
clf = Lasso(alpha=10.0, random_state=13)
clf.fit(X_train, y_train)

y_pred_val_lasso = clf.predict(X_val)

round(np.sqrt(mean_squared_error(y_val, y_pred_val_lasso)), 5)

8426.97894

Let's compare the results on the validation set of the trained models:

* XGBoost
* LightGBM
* CatBoost
* Lasso

In [20]:
models = ['xgb', 'lgb', 'cb', 'lasso']
results = [y_pred_val_xgb, y_pred_val_lgbm, y_pred_val_cat, y_pred_val_lasso]
accuracies = [np.sqrt(mean_squared_error(y_val, result)) for result in results]

min(zip(models, accuracies), key=lambda x: x[1])[0]

'lgb'

So, the LightGBM model has the best RMSE value on the validation set.


---

Finally, let's move to blending the models that we obtained. First, let's calculate the predictions for the trained models on the validation set and concatenate them into a single dataframe `X_val_blend`.

In [21]:
d = {
    'xgb': y_pred_val_xgb,
    'lgb': y_pred_val_lgbm,
    'cb': y_pred_val_cat,
    'lasso': y_pred_val_lasso
}

X_val_blend = pd.DataFrame(d)

In [22]:
X_val_blend.head()

,xgb,lgb,cb,lasso
0,2298.947754,3745.689944,3680.924182,4270.039931
1,3208.189209,5295.171499,4487.549790,6755.853939
2,1171.030029,2362.077406,2899.190806,960.707930
3,1715.524292,2979.686550,3102.992450,3280.292136
4,1780.428223,3058.667040,3404.989586,1586.863807


Let's obtain a matrix of pairwise Pearson Correlation Coefficient (PCC) values for the column of the dataframe.

In [23]:
X_val_blend.corr()

,xgb,lgb,cb,lasso
xgb,1.000000,0.633484,0.776625,0.481110
lgb,0.633484,1.000000,0.844554,0.761926
cb,0.776625,0.844554,1.000000,0.700703
lasso,0.481110,0.761926,0.700703,1.000000


Let's blend models into the ensemble with the weights 0.25, 0.25, 0.25 and 0.25 (just mean value of the predictions).

Compare it with RMSE of each model and think whether this is a good ensemble.

In [24]:
round(np.sqrt(mean_squared_error(y_val, X_val_blend.mean(axis=1))), 5)

8439.26508

The RMSE of such ensemble calculated on the validation set is quite similar to the RMSE of each separate model.

Let's tune the weights of the ensemble. In order to do that, let's run each model weight through `np.linspace(0, 1, 101)`, so that all possible values of each weight will be [0.0, 0.01, ..., 0.99, 1.0]. So, we skip each combinations of weights, if their sum is not equal to 1.0. If the sum of the weights in the combination is equal to 1.0, though, we get ensemble prediction on the validation set using these weights and calculate RMSE value.

In [25]:
from itertools import permutations


weights_list = [np.array(weights) for weights in permutations(np.linspace(0, 1, 101), 4) if sum(weights) == 1.0]

In [26]:
weights_rmse_list = []

for weights in weights_list:
    weights_rmse_list.append((
        weights,
        np.sqrt(mean_squared_error(y_val, np.sum(X_val_blend * weights, axis=1)))
        ))

min(weights_rmse_list, key=lambda x: x[1])

(array([0.  , 0.52, 0.01, 0.47]), 8405.82118305431)

Let's select a combination of weights with the best RMSE value - these are the best weights for the ensemble. 

In [27]:
best_weights = min(weights_rmse_list, key=lambda x: x[1])[0]

Now, the RMSE value of the ensemble is better than the RMSE values of the separate models in it.

Using the best weights obtained above, let's run the best ensemble on the test set.

In [28]:
test_d = {
    'xgb': xgb_reg.predict(X_test),
    'lgb': lgbm_reg.predict(X_test[important_features]),
    'cb': cat_reg.predict(X_test),
    'lasso': clf.predict(X_test)
}
X_test_blend = pd.DataFrame(test_d)

round(np.sqrt(mean_squared_error(y_test, np.sum(X_test_blend * best_weights, axis=1))), 5)

8445.30991